In [1]:
import os
os.chdir('../..')

In [2]:
import pandas as pd

In [3]:
import pandas as pd
segment_df = pd.read_excel('assets/keywords_raw/unspsc.xlsx', sheet_name='Segment', index_col=None, header=None, skiprows=1, usecols="A:B")
family_df = pd.read_excel('assets/keywords_raw/unspsc.xlsx', sheet_name='Family', index_col=None, header=None,skiprows=1, usecols="A:B")
class_df = pd.read_excel('assets/keywords_raw/unspsc.xlsx', sheet_name='Class', index_col=None, header=None,skiprows=1, usecols="A:B")
commodity_df = pd.read_excel('assets/keywords_raw/unspsc.xlsx', sheet_name='Commodity', index_col=None, header=None,skiprows=1, usecols="A:B")

In [5]:
segment_df['parent'] = ''
family_df['parent'] = family_df[0].astype(str).str[:2] + '000000'
class_df['parent'] = class_df[0].astype(str).str[:4]+ '0000'
commodity_df['parent'] = commodity_df[0].astype(str).str[:6]+ '00'

In [6]:
df = pd.concat([segment_df, family_df, class_df, commodity_df])
df.columns = ['code', 'description', 'parent']

In [8]:
import numpy as np
df = df.replace(np.NaN, pd.NA).where(df.notnull(), None)

In [46]:
from typing import List, Optional
from dataclasses import dataclass, field

@dataclass
class CPANode:
    code: str
    description: str
    parent: Optional['CPANode']
    children: List['CPANode'] = field(default_factory=lambda: [])

    def __repr__(self):
        return f'{{code:{self.code}|parent:{self.parent.code if self.parent else "."}|children:{self.children}}}'

In [47]:
def search_nodes(nodes, parent_code, level=0) -> CPANode:
    for node in nodes:
        print(parent_code, node.code)
        if parent_code == node.code:
            return node
        else:
            return search_nodes(node.children, parent_code, level=level+1)

In [57]:
df[df.code == 10100000]

,code,description,parent
0,10100000,Live animals,10000000


In [58]:
nodes = {}
for row in df.sort_values('parent').itertuples():
    # print(row)
    if not row.parent:
        node = CPANode(code=row.code, description=row.description, parent=None)
        nodes[node.code] = node
        continue
    
    node = CPANode(row.code, row.description, nodes[int(row.parent)])
    nodes[node.code] = node
    nodes[int(row.parent)].children.append(node)

In [60]:
def get_parent_path(node, path=[]):
    parent = node.parent
    if not parent:
        return [f'{node.code}_{node.description}'] + path
    else:
        return get_parent_path(node.parent, [f'{node.code}_{node.description}'] + path)

In [77]:
from pathlib import Path
import re


def create_taxonomy(nodes, at_level=1):
    for node in nodes:
        if not node.children:
            path = get_parent_path(node.parent)
            out_path = Path(
                "assets/keywords_raw/Products & Activities/UNSPSC/" + "/".join(path[:at_level+1])
            )
            out_path.mkdir(parents=True, exist_ok=True)
            f_out = Path(out_path / "PRODUCT.tsv.unchecked")
            with f_out.open("a") as f:
                f.write(node.description + "\n")


In [78]:
create_taxonomy(nodes.values())